In [ ]:
import socket

UDP_IP = "192.168.178.55"  # IP-Adresse deines ESP32
UDP_PORT = 6454

# # Erzeuge 513 Byte: 1 Startcode (0) + 512 Kanalwerte (z. B. alle auf 255)
# # dmx_data = bytes([0] + [255] * 512)
# dmx_data = bytes([0, 5] + [0]*511)  # Kanal 1 = 128

# # sock.sendto(b'1:128,2:64,3:255', (ESP32_IP, UDP_PORT))
# sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# sock.sendto(dmx_data, (UDP_IP, UDP_PORT))



dmx_data = bytearray(513)
dmx_data[10] = 5  # Setzt Kanal 3 auf Wert 5

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.sendto(dmx_data, ("192.168.178.55", 6454))
print("DMX-Daten gesendet")


In [ ]:
#kanal 1 auf 128 setzen

dmx_data = bytes([0, 128] + [0]*511)  # Kanal 1 = 128

In [ ]:
import socket

UDP_IP = "192.168.178.55"  # IP-Adresse deines ESP32
UDP_PORT = 6454

def send_dmx_command(command: str):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.sendto(command.encode(), (UDP_IP, UDP_PORT))
    print(f"Gesendet: {command}")

# Beispiele für Kommandos:

# # Direktwert 129 auf Kanal 2
# send_dmx_command("DMXP5#0#2")
# send_dmx_command("DMXC9#150#2")
send_dmx_command("DMXP9#0")

# Prozentwert 55% auf Kanal 3, Geschwindigkeit 1
# send_dmx_command("DMXP4#55#1")

# # Prozentwert 56% auf Kanal 3, Geschwindigkeit 2
# send_dmx_command("DMXP3#56#2#2")

# # RGB-Wert: R=12, G=66, B=3 -> R + G*1000 + B*1000000
rgb_value = 150 + 19*1000 + 12*1000000  # = 3066012
# send_dmx_command(f"DMXR7#{rgb_value}#2")
# send_dmx_command(f"DMXR10#{rgb_value}#2")
# send_dmx_command(f"DMXR10#{rgb_value}#1#1")

# # Tunable White (Typ V): 70% auf Kanal 5 (WW@5, CW@6)
# send_dmx_command("DMXV5#70")

# # Tunable White (Typ W): 70% auf Kanal 7 (CW@7, WW@8)
send_dmx_command("DMXW10#011020#30")

# # Du kannst auch eine Liste von Kommandos senden:
# commands = [
#     "DMXC1#255",   # Kanal 1 auf 255 setzen
#     "DMXP2#50",    # Kanal 2 auf 50%
#     "DMXR10#3066012",  # RGB auf Kanal 10
# ]

# for cmd in commands:
#     send_dmx_command(cmd)


In [ ]:
#Rest API Python Beispiel
import requests
import json

ESP32_IP = "udp2dmx"  # <– hier deine ESP32-IP eintragen

# # Einzelwert ändern (PATCH):
# patch_data = {
#     "ct_config": {
#         "13": 2400,
#         "14": 6500
#     }
# }
# response = requests.post(f"http://{ESP32_IP}/config/patch", json=patch_data)
# print("PATCH Einzelwert:", response.status_code, response.text)

# # MinMax-Werte ändern (PATCH):
# patch_minmax = {
#     "default_ct": {
#         "min": 3000
#     }
# }
# response = requests.post(f"http://{ESP32_IP}/config/patch", json=patch_minmax)
# print("PATCH MinMax:", response.status_code, response.text)


#Beispiel längere Json Datei
patch_minmax = {
    "ct_config": {
        "1": 2200,
        "2": 6400,
        "5": 3300
    },
    "default_ct": {
        "min": 3400,
        "max": 6600
    }
}
patch_minmax = {
    "default_ct": {
        "min": 3400,
        "max": 6600
    }
}
response = requests.post(f"http://{ESP32_IP}/config/patch", json=patch_minmax)
print("PATCH MinMax:", response.status_code, response.text)



# # Ganze Datei lesen (GET):
# response = requests.get(f"http://{ESP32_IP}/config")
# print("GET Config:", response.status_code)
# print(response.json())

# # Ganze Datei ersetzen (POST):
# with open("main\components\spiffs_image\spiffs\config.json", "r") as f:
#     config_data = json.load(f)

# response = requests.post(f"http://{ESP32_IP}/config", json=config_data)
# print("POST Full Config:", response.status_code, response.text)


PATCH MinMax: 200 OK


In [ ]:
#Beispielaufrufe für die REEST API:

# Einzelwert ändern:
curl -X POST http://<ESP32-IP>/config/patch \
     -H "Content-Type: application/json" \
     -d '{"ct_config": {"10": 5100}}'

# MinMax Werte ändern
curl -X POST http://<ESP32-IP>/config/patch \
     -H "Content-Type: application/json" \
     -d '{"default_ct": {"min": 3000}}'

# Ganze Datei lesen:
curl http://<ESP32-IP>/config

#Ganze Datei ersetzen/schreiben:
curl -X POST http://<ESP32-IP>/config \
     -H "Content-Type: application/json" \
     -d @ct_config.json